In [1]:
from gamesopt.train_distributed import train, TrainDistributedConfig
from gamesopt.games import GameOptions, QuadraticGameConfig, GameType
from gamesopt.optimizer import OptimizerOptions, OptimizerType
from gamesopt.aggregator import AggregationOptions, AggregatorType
from gamesopt.attacks import AttackOptions, AttackType
import matplotlib.pyplot as plt

%matplotlib widget
%load_ext autoreload
%autoreload 1

In [2]:
n_iter = 300

n_total = 20
n_byzan = 9

num_samples = 1000
dim = 100
bias = True
sigma = 1e-3  # actually scales sigma
mu = 1e-2
ell = 1e2

attack_type = AttackType.ALIE
n_attacking = 9
ipm_epsilon = 1e-2
rn_sigma = 10.

use_bucketing = True
bucketing_s = 2
aggregator_type = AggregatorType.Mean
trimmed_mean_b = 10
krum_m = 2
clipping_tau = 102
clipping_n_iter = 1
rfa_T: int = 10
rfa_nu: int = 0.1

optimizer_type = OptimizerType.SGDACC
alpha = 0.
lr = 1/2/ell
batch_size = 1
sigmaC = 100.

aggregation_options = AggregationOptions(n_total=n_total,
                                         n_byzan=n_byzan,
                                         aggregator_type=aggregator_type,
                                         use_bucketing=use_bucketing,
                                         bucketing_s=bucketing_s,
                                         trimmed_mean_b=trimmed_mean_b,
                                         krum_m=krum_m,
                                         clipping_tau=clipping_tau,
                                         clipping_n_iter=clipping_n_iter,
                                         rfa_T=rfa_T,
                                         rfa_nu=rfa_nu)

quadratic_game_options = QuadraticGameConfig(num_samples=num_samples,
                                             dim=dim,
                                             num_players=2,
                                             bias=bias,
                                             sigma=sigma,
                                             mu=mu,
                                             ell=ell)

game_options = GameOptions(GameType.QUADRATIC, quadratic_game_options)

attack_options = AttackOptions(n_total=n_total,
                               n_byzan=n_byzan,
                               n_attacking=n_attacking,
                               attack_type=attack_type,
                               ipm_epsilon=ipm_epsilon,
                               rn_sigma=rn_sigma)

optimizer_options = OptimizerOptions(optimizer_type=optimizer_type,
                                     lr=lr,
                                     batch_size=batch_size,
                                     aggregation_options=aggregation_options,
                                     attack_options=attack_options,
                                     sigmaC=sigmaC)

config = TrainDistributedConfig(game=game_options,
                                optimizer=optimizer_options,
                                num_iter=n_iter,
                                n_process=n_total)
print(config)

TrainDistributedConfig(game=GameOptions(game_type=<GameType.QUADRATIC: 'quadratic'>, quadratic_options=QuadraticGameConfig(num_samples=1000, dim=100, num_players=2, bias=True, sigma=0.001, mu=0.01, ell=100.0, seed=0, matrix=None)), optimizer=OptimizerOptions(optimizer_type=<OptimizerType.SGDACC: 'SGDACC'>, lr=0.005, batch_size=1, aggregation_options=AggregationOptions(n_total=20, n_byzan=9, aggregator_type=<AggregatorType.Mean: 'Mean'>, use_bucketing=True, bucketing_s=2, clipping_tau=102, clipping_n_iter=1, trimmed_mean_b=10, krum_m=2, rfa_T=10, rfa_nu=0.1), attack_options=AttackOptions(n_total=20, n_byzan=9, n_attacking=9, attack_type=<AttackType.ALIE: 'ALittleIsEnough'>, rn_sigma=10.0, ipm_epsilon=0.01, alie_z=None), sigmaC=100.0), num_iter=300, n_process=20)


In [ ]:
%time record = train(config)

Trying port 29497
Matrix generated
Bias generated
Solution found


In [ ]:
plt.figure()
plt.yscale('log')
record.load_metrics()
# plt.figure()
plt.plot(record.metrics.num_grad, record.metrics.dist,
         label="%s %s lr=%.e" % (config.optimizer.optimizer_type.value,
                                 config.optimizer.aggregation_options.aggregator_type.value,
                                 config.optimizer.lr))
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
record.metrics.dist